In [ ]:
!pip install keras_applications
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#flatten reduces dimension of data to a unidimensional array
#The Dense layer takes the input tensor, multiplies it with a weight matrix, adds a bias term, (Bias is the difference between predicted values and expected results.)
#and applies an activation function to produce the output tensor (tensor=data container, A vector is a 1D tensor, a matrix is a 2D tensor.)

# Define the data directories and hyperparameters
train_dir = '/content/drive/MyDrive/main set/train'
validation_dir = '/content/drive/MyDrive/main set/val'
test_dir = '/content/drive/MyDrive/main set/test'
img_size = (64, 64)
num_epochs = 100
batch_size = 32

# print("Number of training samples:", len(train_generator.filenames))
# print("Number of validation samples:", len(validation_generator.filenames))

# Create data generators for the training, validation, and test data
train_datagen = ImageDataGenerator(
    rescale=1./255, #to make the values fall in the range [0,1] (normalization)
    rotation_range=20, #images will be rotated randomly between -20 to +20 degrees during training.
    width_shift_range=0.2, #image can be horizontally shifted by up to 20% of its total width. 
    height_shift_range=0.2,
    shear_range=0.2, # shearing transformations to the image. (slants the shape of an object)
    zoom_range=0.2, #random zooming to the image
    horizontal_flip=True, 
    fill_mode='nearest') #fills gaps in distortion with nearest image

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#flow_from_dir creates an iterator that yields batches of images and their corresponding labels, 
#which can then be used to train the CNN model.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', #this sets the value of the vector in each image (0 for secondary and 1 for actual wanted one(1,0))
    shuffle=False)


#top layer is exculded cuz By excluding the top layer of the pre-trained model, we remove the classification layer and replace it with a new layer that is compatible with the new classification task. This allows us to leverage the feature extraction capabilities of the pre-trained model while adapting the final classification layer to the new problem.
# Load the pre-trained ResNet-50 model and freeze its weights
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3)) #imagenet = model is initialized with pre-trained weights on the ImageNet dataset, top layer of pretrained model shouldnt be included, channels is 3, representing the RGB color channels
for layer in resnet.layers:
    layer.trainable = False #these layers will keep their pre-trained weights and will only be used for feature extraction, while the new custom output layer is trained using the updated gradients. (freeze weights bascically)

# Add a custom output layer to the ResNet-50 model
x = Flatten()(resnet.output) #converts it to a 1D array
x = Dense(128, activation="relu")(x) #layer with 128 neurons, relu applies a non-linear transformation to the input data and outputs a new set of values which allows it to learn more complex patterns and relationships in the data.(rectified linear unnit) (Increasing the number of units in the dense layer may improve the performance of the model, but it also increases the number of parameters and the risk of overfitting.)
output = Dense(2, activation="softmax")(x) #2 neurons and "softmax" activation function which produces a probability distribution over the two possible classes.
model = Model(inputs=resnet.input, outputs=output)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with the generators
history = model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

# Output the training and validation accuracy during training
print("best Training accuracy:", max(history.history['accuracy']))
print("best Validation accuracy:", max(history.history['val_accuracy']))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Save the model
model.save('green_brown_land_model.h5') # a data file saved in the Hierarchical Data Format (HDF). It contains multidimensional arrays of scientific data.

# Predict the class of the test images
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1) #numpy function that takes in the probability predictions and returns the indices of the highest values along axis 1, which represents the class with the highest predicted probability.

# Print the predicted class and image label for each test image
for i in range(len(predicted_classes)):
    img_path = test_generator.filepaths[i]
    img = image.load_img(img_path, target_size=img_size)
    if predicted_classes[i] == 0:
        print(f"{img_path} contains brown land. Prediction: brown land.")
    else:
        print(f"{img_path} contains green land. Prediction: green land.")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found 304 images belonging to 2 classes.
Found 308 images belonging to 2 classes.
Found 44 images belonging to 2 classes.
Epoch 1/100
10/10 [==============================] - 194s 21s/step - loss: 0.8510 - accuracy: 0.4803 - val_loss: 0.7592 - val_accuracy: 0.5000
Epoch 2/100
10/10 [==============================] - 10s 1s/step - loss: 0.6898 - accuracy: 0.4934 - val_loss: 0.6597 - val_accuracy: 0.8766
Epoch 3/100
10/10 [==============================] - 9s 954ms/step - loss: 0.6690 - accuracy: 0.5921 - val_loss: 0.6409 - val_accuracy: 0.9123
Epoch 4/100
10/10 [==============================] - 9s 961ms/step - loss: 0.6558 - accuracy: 0.6053 - val_loss: 0.6248 - val_accuracy: 0.7435
Epoch 5/100
10/10 [==============================] - 9s 959ms/step - loss: 0.6149 - accuracy: 0.8553 - val_loss: 0.6092 - val_accuracy: 0.6948
Epoch 6/100
10/10 [==============================] - 10s 1s/step -

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras_applications
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#flatten reduces dimension of data to a unidimensional array
#The Dense layer takes the input tensor, multiplies it with a weight matrix, adds a bias term, (Bias is the difference between predicted values and expected results.)
#and applies an activation function to produce the output tensor (tensor=data container, A vector is a 1D tensor, a matrix is a 2D tensor.)

# Define the data directories and hyperparameters
train_dir = '/content/drive/MyDrive/soil/soil testing/Train'
validation_dir = '/content/drive/MyDrive/soil/soil testing/val'
test_dir = '/content/drive/MyDrive/soil/soil testing/test'
img_size = (64, 64)
num_epochs = 25
batch_size = 32

# print("Number of training samples:", len(train_generator.filenames))
# print("Number of validation samples:", len(validation_generator.filenames))

# Create data generators for the training, validation, and test data
train_datagen = ImageDataGenerator(
    rescale=1./255, #to make the values fall in the range [0,1] (normalization)
    rotation_range=20, #images will be rotated randomly between -20 to +20 degrees during training.
    width_shift_range=0.2, #image can be horizontally shifted by up to 20% of its total width. 
    height_shift_range=0.2,
    shear_range=0.2, # shearing transformations to the image. (slants the shape of an object)
    zoom_range=0.2, #random zooming to the image
    horizontal_flip=True, 
    fill_mode='nearest') #fills gaps in distortion with nearest image

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#flow_from_dir creates an iterator that yields batches of images and their corresponding labels, 
#which can then be used to train the CNN model.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', #this sets the value of the vector in each image (0 for secondary and 1 for actual wanted one(1,0))
    shuffle=False)


#top layer is exculded cuz By excluding the top layer of the pre-trained model, we remove the classification layer and replace it with a new layer that is compatible with the new classification task. This allows us to leverage the feature extraction capabilities of the pre-trained model while adapting the final classification layer to the new problem.
# Load the pre-trained ResNet-50 model and freeze its weights
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3)) #imagenet = model is initialized with pre-trained weights on the ImageNet dataset, top layer of pretrained model shouldnt be included, channels is 3, representing the RGB color channels
for layer in resnet.layers:
    layer.trainable = False #these layers will keep their pre-trained weights and will only be used for feature extraction, while the new custom output layer is trained using the updated gradients. (freeze weights bascically)

# Add a custom output layer to the ResNet-50 model
x = Flatten()(resnet.output) #converts it to a 1D array
x = Dense(128, activation="relu")(x) #layer with 128 neurons, relu applies a non-linear transformation to the input data and outputs a new set of values which allows it to learn more complex patterns and relationships in the data.(rectified linear unnit) (Increasing the number of units in the dense layer may improve the performance of the model, but it also increases the number of parameters and the risk of overfitting.)
output = Dense(2, activation="softmax")(x) #2 neurons and "softmax" activation function which produces a probability distribution over the two possible classes.
model = Model(inputs=resnet.input, outputs=output)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with the generators
history = model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

# Output the training and validation accuracy during training
print("best Training accuracy:", max(history.history['accuracy']))
print("best Validation accuracy:", max(history.history['val_accuracy']))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Save the model
model.save('green_brown_land_model.h5') # a data file saved in the Hierarchical Data Format (HDF). It contains multidimensional arrays of scientific data.

# Predict the class of the test images
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1) #numpy function that takes in the probability predictions and returns the indices of the highest values along axis 1, which represents the class with the highest predicted probability.

# Print the predicted class and image label for each test image
for i in range(len(predicted_classes)):
    img_path = test_generator.filepaths[i]
    img = image.load_img(img_path, target_size=img_size)
    if predicted_classes[i] == 0:
        print(f"{img_path} contains black soil. Prediction: cotton.")
    else:
        print(f"{img_path} contains yellow soil. Prediction: mango.")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found 310 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/25
10/10 [==============================] - 18s 1s/step - loss: 0.7431 - accuracy: 0.5548 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 2/25
10/10 [==============================] - 12s 1s/step - loss: 0.6559 - accuracy: 0.6903 - val_loss: 0.6216 - val_accuracy: 0.8438
Epoch 3/25
10/10 [==============================] - 12s 1s/step - loss: 0.6188 - accuracy: 0.7000 - val_loss: 0.5883 - val_accuracy: 0.8438
Epoch 4/25
10/10 [==============================] - 13s 1s/step - loss: 0.5615 - accuracy: 0.7903 - val_loss: 0.5638 - val_accuracy: 0.8500
Epoch 5/25
10/10 [==============================] - 12s 1s/step - loss: 0.5474 - accuracy: 0.7871 - val_loss: 0.5948 - val_accuracy: 0.7125
Epoch 6/25
10/10 [==============================] - 16s 2s/step - loss: 0.5277